#### Create CSV File from given table

In [ ]:
import pandas as pd

# Define your Q&A pairs
faq_data = {
    "Question": [
        "What are your working hours?",
        "How can I reset my password?",
        "What is the refund policy?",
        "Do you offer customer support?"
        ],
    "Answer": [
        "Our working hours are 9 AM to 6 PM, Monday to Friday.",
        "You can reset your password using the ‘Forgot Password’ option on the login page.",
        "Refunds can be requested within 30 days of purchase with a valid receipt.",
        "Yes, customer support is available 24/7 via phone, email, and chat."
        ]
}

faq_df = pd.DataFrame(faq_data)
faq_df.to_csv("faqs.csv", index=False)

#### Install Necessary Libraries from requirement file

In [ ]:
!pip install -r requirements.txt

#### After Installing Restart the session

#### Import all necessary Libraries

In [ ]:
import gradio as gr
import pandas as pd
import os
import re
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import google.generativeai as genai

#### Set up Google API Key

In [ ]:
os.environ["GOOGLE_API_KEY"] = "enter Your google api Key"  # google api Key
API_KEY = os.environ["GOOGLE_API_KEY"]
genai.configure(api_key=API_KEY)

#### Initialize embeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


#### function to clean the txt

In [ ]:
def normalize_text(text):
  text = str(text).strip().lower()
  text = re.sub(r"\s+", " ", text)
  text = re.sub(r"[^\w\s]", "", text)
  return text

#### function to load the data

In [ ]:
def load_data(file):

  qa_pairs = [] # list to store all cleaned data from the file
  filename = file.name.lower()

  # file type
  if filename.endswith(".csv"): # If file is csv
      df = pd.read_csv(file.name)
  elif filename.endswith((".xls", ".xlsx")): # if file type is Excel
      df = pd.read_excel(file.name)
  else:
      raise ValueError("Upload CSV or Excel")

  # Required Columns
  if 'Question' not in df.columns or 'Answer' not in df.columns:
      raise ValueError("file needs to have 'Question' and 'Answer' columns.")

  # store Normalize question answer pairs
  for _, row in df.iterrows():
      question = normalize_text(row["Question"])
      answer = str(row["Answer"]).strip()
      if question and answer:
          qa_pairs.append((question, answer))

  return qa_pairs


#### FAQ Vectorization

In [ ]:
def build_vector_store(qa_pairs):
  question = [q for q,a in qa_pairs]
  metadata = [{"question": q, "answer": a} for q, a in qa_pairs]
  vector_store = FAISS.from_texts(question, embedding=embeddings, metadatas=metadata)
  vector_store.save_local("faiss_index")


#### setting up the AI assistant for answering questions using your FAQ knowledge base.

In [ ]:
def create_qa_chain():
  prompt_text = """
  You are a helpful assistant. Use the context of FAQs below to answer the user's question.

  Only answer using the provided FAQ content.
  If you cannot find the answer, reply:
  "I'm sorry, the answer to this question is not available in the provided FAQs. Please connect with an agent."

  If the answer has multiple points, present them as bullet points.

  FAQs Context:
  {context}

  User Question:
  {question}

  Answer:
  """

  prompt = PromptTemplate(template=prompt_text, input_variables=["context", "question"])
  qa_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3, google_api_key=API_KEY)
  qa_chain = load_qa_chain(qa_model, chain_type="stuff", prompt=prompt)
  return qa_chain

#### Answer user query

In [ ]:
def handle_user_question(user_question_raw):
  if not os.path.exists("faiss_index"):
      return "Please upload and process files first(Csv or excel)."

  user_question = normalize_text(user_question_raw)  # Clean user input
  vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True) # Load the FAISS vector store

  faqs = vector_store.similarity_search(user_question, k=3)  # Retrieve top 3 similar FAQs
  #If no relevant match in top 3, try top 5
  if not faqs:
    faqs = vector_store.similarity_search(user_question, k=5)
  # If no match found
  if not faqs:
    return "No relevant information was found in the uploaded files."

  # Prepare the retrieved FAQs for the QA chain
  faq_entries = []
  for faq in faqs:
      q = faq.metadata.get("question", "").strip()
      a = faq.metadata.get("answer", "").strip()
      content = f"Q: {q}\nA: {a}"
      faq_entries.append(Document(page_content=content))

  chain = create_qa_chain()
  result = chain({"input_documents": faq_entries, "question": user_question}, return_only_outputs=True)
  answer = result["output_text"].strip()

  if "i don't know" in answer.lower() or len(answer) < 5:
      return "answer to this question is not available in the provided files."
  else:
      return answer

#### Uploaded File Processing

In [ ]:
def process_files(file):
  try:
      qa_pairs = load_data(file)
      build_vector_store(qa_pairs)
      return "Files processed and knowledge base created."
  except Exception as e:
      return f"Error: {e}"


#### Build UI using Gradio

In [ ]:
with gr.Blocks() as chatbot_ui:
  gr.Markdown("## AI-Powered FAQ Assistant :: Ask Questions from CSV/Excel Q&A")

  with gr.Row():
      with gr.Column():
          file_uploader = gr.File(file_types=[".xlsx", ".csv"], type="file", label="Upload CSV/Excel File")
          process_button = gr.Button("Process Files")
          status_output = gr.Textbox(label="Status")

      with gr.Column():
          user_question_input = gr.Textbox(label="Ask your question")
          answer_output = gr.Markdown()

  process_button.click(process_files, inputs=file_uploader, outputs=status_output)
  user_question_input.submit(handle_user_question, inputs=user_question_input, outputs=answer_output)

chatbot_ui.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.41.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://dda01e80c5f53764ff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
